# Let's look at the raw data, specifically the washover stats

In [1]:
# Import packages we need

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,shutil

In [2]:
#load the labels
#round 2 labels
#labeler_data = pd.read_csv('../data/tables/labels/Priority_1604952200293.csv')
#round 3 labels
#labeler_data = pd.read_csv('../data/tables/labelerdata/Priority_1613057474309.csv')
#round 4 labels
#labeler_data = pd.read_csv('../data/tables/labelerdata/Priority_1618341223365.csv')
#round 5 labels
labeler_data = pd.read_csv('../data/tables/labelerdata/Priority_1629556104483.csv')


## Get data into shape for release

In [3]:
# make data into final zenodo form
releaseData = labeler_data
releaseData = releaseData.drop(['_id','catalogId', 'archiveId','date','catalog','additionalComments'], axis = 1)
releaseData = releaseData.drop(['imageId'], axis = 1)
releaseData = releaseData.rename({'water': 'allWater'}, axis=1)
releaseData.head()

,userId,archive,image,allWater,devType,washoverType,dmgType,impactType,terrianType
0,5f4c002a9cdb9b001675a8e2,Florence,P26058806.jpg,True,NaN,NaN,NaN,NaN,NaN
1,5f4c002a9cdb9b001675a8e2,Michael,S28468531.jpg,NaN,developed,noWashover,noDamage,NaN,"marsh,inland,river"
2,5f4c002a9cdb9b001675a8e2,Michael,S28466899.jpg,NaN,developed,noWashover,damage,NaN,inland
3,5f4c002a9cdb9b001675a8e2,Isaias,S21773987.jpg,NaN,developed,noWashover,noDamage,swash,sandyCoastline
4,5f4c002a9cdb9b001675a8e2,Florence,P26057665.jpg,NaN,undeveloped,noWashover,noDamage,"swash,collision",sandyCoastline


In [4]:
# replace NaN in devType, washoverType, dmgType,
values = {'allWater':'False', 'devType': 'undeveloped', 'washoverType': 'noWashover', 'dmgType': 'noDamage'}

releaseData = releaseData.fillna(value=values)
releaseData

,userId,archive,image,allWater,devType,washoverType,dmgType,impactType,terrianType
0,5f4c002a9cdb9b001675a8e2,Florence,P26058806.jpg,True,undeveloped,noWashover,noDamage,NaN,NaN
1,5f4c002a9cdb9b001675a8e2,Michael,S28468531.jpg,False,developed,noWashover,noDamage,NaN,"marsh,inland,river"
2,5f4c002a9cdb9b001675a8e2,Michael,S28466899.jpg,False,developed,noWashover,damage,NaN,inland
3,5f4c002a9cdb9b001675a8e2,Isaias,S21773987.jpg,False,developed,noWashover,noDamage,swash,sandyCoastline
4,5f4c002a9cdb9b001675a8e2,Florence,P26057665.jpg,False,undeveloped,noWashover,noDamage,"swash,collision",sandyCoastline
...,...,...,...,...,...,...,...,...,...
7872,6050c31a1bd6510018fa2a87,Florence5,P26054287.jpg,False,developed,noWashover,damage,swash,"sandyCoastline,marsh"
7873,6050c31a1bd6510018fa2a87,Florence5,P26056655.jpg,False,developed,washover,damage,overwash,"sandyCoastline,marsh,estuary"
7874,6050c31a1bd6510018fa2a87,Florence5,P26057080.jpg,False,developed,washover,damage,"collision,overwash","sandyCoastline,marsh,estuary"
7875,6050c31a1bd6510018fa2a87,Florence5,P26056220.jpg,False,developed,noWashover,damage,collision,"sandyCoastline,marsh,estuary"


In [5]:
releaseData.shape

(7877, 9)

In [6]:
Threshold = releaseData[["userId","archive","image"]].groupby(['userId', 'archive']).count().reset_index()
#Threshold = Threshold[Threshold["image"] == 100].reset_index()
Threshold

,userId,archive,image
0,5f4c002a9cdb9b001675a8e2,Barry5,100
1,5f4c002a9cdb9b001675a8e2,Delta5,100
2,5f4c002a9cdb9b001675a8e2,Dorian5,100
3,5f4c002a9cdb9b001675a8e2,Florence,100
4,5f4c002a9cdb9b001675a8e2,Florence3,100
...,...,...,...
85,6065fb60693eea0016eea0c1,Isaias4,100
86,609ea7ef4a006500165c5de4,Dorian5,100
87,609ea7ef4a006500165c5de4,Isaias5,100
88,609ea7ef4a006500165c5de4,Zeta5,100


In [7]:
#add a true column
Threshold.rename(columns = {"image": "total"}, inplace=True)

#join Threshold with releaseData
releaseData = releaseData.merge(Threshold, left_on = ['userId', 'archive'], right_on = ['userId', 'archive'])

# remove rows where total <100
releaseData = releaseData[releaseData["total"] == 100]
releaseData

,userId,archive,image,allWater,devType,washoverType,dmgType,impactType,terrianType,total
0,5f4c002a9cdb9b001675a8e2,Florence,P26058806.jpg,True,undeveloped,noWashover,noDamage,NaN,NaN,100
1,5f4c002a9cdb9b001675a8e2,Florence,P26057665.jpg,False,undeveloped,noWashover,noDamage,"swash,collision",sandyCoastline,100
2,5f4c002a9cdb9b001675a8e2,Florence,P26059250.jpg,False,developed,washover,noDamage,"swash,collision,overwash",sandyCoastline,100
3,5f4c002a9cdb9b001675a8e2,Florence,P26060067.jpg,False,undeveloped,noWashover,noDamage,"swash,collision",sandyCoastline,100
4,5f4c002a9cdb9b001675a8e2,Florence,P26057539.jpg,False,undeveloped,washover,noDamage,"swash,collision,overwash",sandyCoastline,100
...,...,...,...,...,...,...,...,...,...,...
7872,5f4c002a9cdb9b001675a8e2,Michael5,S28468807.jpg,False,developed,noWashover,damage,NaN,inland,100
7873,5f4c002a9cdb9b001675a8e2,Michael5,S28464064.jpg,False,undeveloped,noWashover,noDamage,NaN,marsh,100
7874,5f4c002a9cdb9b001675a8e2,Michael5,S28465721.jpg,False,developed,noWashover,damage,NaN,"marsh,inland",100
7875,5f4c002a9cdb9b001675a8e2,Michael5,P28460364.jpg,False,developed,noWashover,noDamage,swash,sandyCoastline,100


In [8]:
#replace user IDs with `User 1`, etc...
releaseData.shape
releaseData.userId.unique()

array(['5f4c002a9cdb9b001675a8e2', '5f77871024f95e0018305349',
       '5f6635628cb36a001698427b', '5f7af56443360b001790063d',
       '5f7c8c4287bff50017610126', '5f89a4957c0f40001652c05d',
       '5f89c25b67d0e30016a5e70a', '5fbc7ccabf106c0017b864a0',
       '5fc7dc859f59bc0016211439', '5f8f5f59e4dac20017f9070e',
       '600994de5eff87001694a117', '5fa4542fec49d20016f772a8',
       '5ff24c3448a4570013d1a9ed', '6050c31a1bd6510018fa2a87',
       '6051072d523d7200169cc5e5', '6052b1a5f441e80013ac154e',
       '6054a37b79f962001798e2eb', '6054b60503d3fc0013d43782',
       '6064ef18340146001681bf0c', '6065fb60693eea0016eea0c1',
       '609ea7ef4a006500165c5de4', '60ac0f1d20620600172dcfd9',
       '5fc94f320dc93600168cf242'], dtype=object)

In [9]:
releaseData['userId'] = pd.factorize(releaseData.userId)[0] + 1
releaseData.userId.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23])

In [10]:
releaseData.head()

,userId,archive,image,allWater,devType,washoverType,dmgType,impactType,terrianType,total
0,1,Florence,P26058806.jpg,True,undeveloped,noWashover,noDamage,NaN,NaN,100
1,1,Florence,P26057665.jpg,False,undeveloped,noWashover,noDamage,"swash,collision",sandyCoastline,100
2,1,Florence,P26059250.jpg,False,developed,washover,noDamage,"swash,collision,overwash",sandyCoastline,100
3,1,Florence,P26060067.jpg,False,undeveloped,noWashover,noDamage,"swash,collision",sandyCoastline,100
4,1,Florence,P26057539.jpg,False,undeveloped,washover,noDamage,"swash,collision,overwash",sandyCoastline,100


In [11]:
releaseData.archive.unique()

array(['Florence', 'Michael', 'Isaias', 'MichaelWash3', 'Isaias3',
       'Florence3', 'FlorenceWash3', 'Michael3', 'IsaiasWash3',
       'Michael4', 'MichaelWash4', 'Florence4', 'IsaiasWash4', 'Isaias4',
       'FlorenceWash4', 'Barry5', 'Delta5', 'Dorian5', 'Gordon5',
       'Laura5', 'Sally5', 'Zeta5', 'Isaias5', 'Florence5', 'Michael5'],
      dtype=object)

In [12]:
#replace column title with NOAA flight
#releaseData = releaseData.rename({'archive': 'NOAA_flight'}, axis=1)

# replace archive with specific flight
#releaseData = releaseData.replace(('Florence','Florence3','FlorenceWash3','Florence4','FlorenceWash4','Florence5'),'Florence_20180917a_jpgs')
#releaseData = releaseData.replace(('Michael','Michael3','MichaelWash3','Michael4','MichaelWash4','Michael5'), 'Michael_20181011a_jpgs')
#releaseData = releaseData.replace(('Isaias','Isaias3','IsaiasWash3','Isaias4','IsaiasWash4','Isaias5'),'Isaias_20200804a_jpgs')


#releaseData.head()

In [13]:
# now to link with the more complicated ones:
# Barry5', 'Delta5', 'Dorian5', 'Gordon5','Laura5', 'Sally5', 'Zeta5', 'Isaias5', 'Florence5', 'Michael5

#read in the master DF
df = pd.read_csv('../data/tables/MasterDF.csv')

# selecting rows based on bbeing labeled
L_df = df[df['labeled'].notnull()]

#select based on the labeled round
#L_df = df[df['labeled'] == 5]

#Any Duplicates?, should be fal;se via: https://stackoverflow.com/questions/50242968/check-for-duplicate-values-in-pandas-dataframe-column
L_df['image'].duplicated().any() 

False

In [14]:
#combine the storm name and archive into the flight tag
L_df["NOAA_flight"] = L_df["storm_id"] + "/" + L_df['archive']

#subset dataframe into just the image and NOAA flight info
L_df = L_df[['image','NOAA_flight']]

<ipython-input-14-6304960fe838>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  L_df["NOAA_flight"] = L_df["storm_id"] + "/" + L_df['archive']


In [15]:
#merge to prediction dataframe
releaseData = pd.merge(releaseData, L_df, on="image", how = "left")
releaseData.head()

,userId,archive,image,allWater,devType,washoverType,dmgType,impactType,terrianType,total,NOAA_flight
0,1,Florence,P26058806.jpg,True,undeveloped,noWashover,noDamage,NaN,NaN,100,florence/20180917a_jpgs
1,1,Florence,P26057665.jpg,False,undeveloped,noWashover,noDamage,"swash,collision",sandyCoastline,100,florence/20180917a_jpgs
2,1,Florence,P26059250.jpg,False,developed,washover,noDamage,"swash,collision,overwash",sandyCoastline,100,florence/20180917a_jpgs
3,1,Florence,P26060067.jpg,False,undeveloped,noWashover,noDamage,"swash,collision",sandyCoastline,100,florence/20180917a_jpgs
4,1,Florence,P26057539.jpg,False,undeveloped,washover,noDamage,"swash,collision,overwash",sandyCoastline,100,florence/20180917a_jpgs


In [16]:
releaseData.shape

(7600, 11)

In [17]:
#save release data
releaseData.to_csv('../data/tables/labelerdata/ReleaseData08222021.csv',na_rep='NaN')